In [1]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

# Set the number of dictionary atoms (components) and sparsity level
# N_components = 3  # Number of Sparse Components
n_components = 10  # Try values like 20, 50, or 100
n_nonzero_coefs = int(n_components * 0.3)

In [2]:
def create_sparse_representation(data, n_components, n_nonzero_coefs):
    """
    Create a sparse representation with reduced dimensions.
    
    Parameters:
    - data: Input data with shape (n_samples, n_data_points, n_features)
    - n_components: Reduced dimensionality
    - n_nonzero_coefs: Sparsity level
    
    Returns:
    - Sparse representation with reduced dimensions.
    """
    n_samples, n_data_points, n_features = data.shape

    # Initialize sparse representation array
    sparse_representation = np.zeros((n_samples, n_components, n_features))

    # Process each feature separately
    for feature in range(n_features):
        # Extract data for current feature
        feature_data = data[:, :, feature]  # Shape: (n_samples, n_data_points)

        # Learn a dictionary (or use a pre-defined one)
        dict_learner = DictionaryLearning(
            n_components=n_components, 
            transform_algorithm='lasso_lars', 
            transform_n_nonzero_coefs=n_nonzero_coefs
        )
        dict_learner.fit(feature_data)
        sparse_codes = dict_learner.transform(feature_data)

        # Store the sparse codes for this feature
        sparse_representation[:, :, feature] = sparse_codes

    return sparse_representation

In [3]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))

# Learn the dictionary
dict_learner = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', transform_n_nonzero_coefs=n_nonzero_coefs, n_jobs= -1)
dict_learner.fit(train_data_combined)
# D = dict_learner.components_  # Shape: (n_components, feature_dim)


train_alice_sparse_codes = dict_learner.transform(train_alice_magnitude)
train_eve_sparse_codes = dict_learner.transform(train_eve_magnitude)
train_sparse_codes = np.vstack((train_alice_sparse_codes, train_eve_sparse_codes))
print(train_alice_sparse_codes.shape)

(800, 10)


In [4]:
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]

# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))


# Transform test data into sparse codes
test_alice_sparse_codes = dict_learner.transform(test_alice_magnitude)
test_eve_sparse_codes = dict_learner.transform(test_eve_magnitude)

# Combine the sparse codes
test_sparse_codes = np.vstack((test_alice_sparse_codes, test_eve_sparse_codes))
print('Test data shape:', test_sparse_codes.shape)

Test data shape: (400, 10)


In [5]:
# Labels for training data
train_alice_labels = np.zeros(train_alice_sparse_codes.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_sparse_codes.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Labels for test data
test_alice_labels = np.zeros(test_alice_sparse_codes.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_sparse_codes.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))


In [6]:
# Dictionary
D = train_sparse_codes.T
print('Dictionary shape:', D.shape)

Dictionary shape: (10, 1600)


In [7]:
# ---------------------------------------------- Sparse Coding & Classification ------------------------------------------------
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=15):
    # print(f'tSample: {tSample.shape}')
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    # Find sparse coefficients for the new signal
    coefficients = find_sparse_coefficients(tSample, D)
    
    
    # Initialize residuals list
    residuals = []

    # Calculate residual for each class
    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class

# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for cir in test_sparse_codes:
    predicted_class = classify_signal(cir, D, train_labels)
    predictions.append(predicted_class)

predictions = np.array(predictions)
print(predictions.shape)


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

(400,)


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Classification Accuracy: 52.00%
tp: 175
tn: 33
fp: 167
fn: 25
MDR: 0.835
FAR: 0.125
AR: 0.52
